In [ ]:
#| default_exp datasets.live

# LIVE

> Building a `tf.data.Dataset` for LIVE.

In [ ]:
#| hide
import os; os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [ ]:
#| export
from pathlib import Path
import pandas as pd
import tensorflow as tf
import cv2

After setting up the path to the directory and loading the corresponding `.csv` file, we need to create a generator that will iterate over the dataframe, load and return a 3-tuple: `(Reference Image, Distorted Image, DMOS)`. When can the pass that generator into a `tf.data.Dataset.from_generator()` to build the `Dataset` object:

In [ ]:
#| export
class LIVE():
    """Builder for the LIVE dataset"""

    def __init__(self,
                 path, # Path to the root directory of the dataset
                 ):
        self.path_root = Path(path) if isinstance(path, str) else path
        self.path_ref = self.path_root/"refimgs"
        self.path_csv = self.path_root/"image_pairs_dmos.csv"
        self.data = pd.read_csv(self.path_csv, index_col=0)

    @property
    def dataset(self):
        """tf.data.Dataset object built from the LIVE dataset."""
        return tf.data.Dataset.from_generator(
                self.data_gen,
                output_signature=(
                    tf.TensorSpec(shape=(512, 768, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(512, 768, 3), dtype=tf.float32),
                    tf.TensorSpec(shape=(), dtype=tf.float32)
                )
            ) 

    def data_gen(self):
        """Dataset generator to build the tf.data.Dataset."""
        for i, (ref, dist, dmos) in self.data.iterrows():
            dist = cv2.imread(str(self.path_root/dist))
            dist = cv2.cvtColor(dist, cv2.COLOR_BGR2RGB)/255.0
            ref = cv2.imread(str(self.path_ref/ref))
            ref = cv2.cvtColor(ref, cv2.COLOR_BGR2RGB)/255.0
            yield ref, dist, dmos

In [ ]:
l = LIVE(path = Path("/media/disk/databases/BBDD_video_image/Image_Quality/LIVE/"))

In [ ]:
for a, b, c in l.dataset:
    break
assert a.shape == b.shape